# Automated Sensing of Red vs. White Wine
_... with the cheapest sensor possible_

#### Data Science 420 BB
#### University of Washington Professional & Continuing Education

#### Homework 1: Low-cost Sensor
#### Leo Salemann, 4/11/18

# Read the Data, Have a Look
## Initial Load

In [ ]:
file <- "./RedWhiteWine.csv"
all_data <- read.csv(file, header=TRUE, sep=",", stringsAsFactors = FALSE)

# If the target column, say column class, is numerical, you need to convert it to factors in R for classification problem
all_data$Class <- factor(data$Class)
head(all_data)


## Reduce to the minimally-required columns

In [ ]:
library("dplyr")

data = select(all_data, fixed.acidity, volatile.acidity, citric.acid, residual.sugar, chlorides, density, Class)
head(data)
dim(data)
summary(data)

## Redefine Class 1 as Red and 0 as white [TODO]

# Create a Naive Bayes Model, Try a Prediction

In [ ]:
nrows <- nrow(data)
smp_size <- floor(0.75 * nrows)

## set the seed to make your partition reproductible
set.seed(8720885)
train_ind <- sample(seq_len(nrows), size = smp_size)

In [ ]:
train <- data[train_ind, ]
test <- data[-train_ind, ]

In [ ]:
options(repr.plot.width=4, repr.plot.height=4)
plot(nbc)

In [ ]:
install.packages("naivebayes")
library(naivebayes)
nbc <- naive_bayes(Class ~ ., data = train)
plot(nbc)

In [ ]:
# Predict the test data
test_pred <- predict(nbc, test,type = "prob")

accuracy <- sum(test[,"Class"]==test_pred)/length(test_pred)
print(paste("Accuracy=", round(accuracy*100,2), "%", sep=""))

# Make a function that quickly tests accuracy

In [69]:
install.packages("naivebayes")
library(naivebayes)

test_nbc_accuracy <- function(df) {
    # split into test and training sets
    nrows <- nrow(data)
    smp_size <- floor(0.75 * nrows)

    set.seed(98027)
    train_ind <- sample(seq_len(nrows), size = smp_size)
    
    train <- data[train_ind, ]
    test <- data[-train_ind, ]
    
    # create the Naive Bayes Classifer (nbc)
    nbc <- naive_bayes(Class ~ ., data = train)
    
    # Make a prediction & test accuracy
    test_pred <- predict(nbc, test,type = "prob")

    accuracy <- sum(test[,"Class"]==test_pred)/length(test_pred)
    print(paste("Accuracy=", round(accuracy*100,2), "%", sep=""))

}

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


In [70]:
test_nbc_accuracy(data)

[1] "Accuracy=0.43%"


In [ ]:
# Try Dropping chlorides, since it's the opposite of residual sugar

In [ ]:
# Try Dropping Volatile acidity

# ==========================< JUNK >===========================

## Pull out a handy plotting function from last quarter

In [ ]:
require(ggplot2)
plot.density = function(df, col, xlabel, title) {
    ggplot(df, aes(col)) + 
          geom_bar() + 
          xlab(xlabel) + 
          ggtitle(title) + 
          theme(plot.title = element_text(hjust = 0.5, size=20, face="bold"),
               axis.title.x = element_text(size=15))
}

In [ ]:
options(repr.plot.width=12, repr.plot.height=4)
plot.density(data, data$Class, "1 = Red?, 2 = White?", "Wine Color")

## Try some Fancy Matrix Plotting

In [ ]:
require(GGally)

plot.matrix = function(df, cols){
    options(repr.plot.width=8, repr.plot.height=8)
    ggpairs(data=df,
            columns=cols,
            upper = list(continuous = "density"),
            lower = list(continuous = "density"),
            cardinality_threshold = 130)
}

In [ ]:
plot.matrix(data, c(1:6,7))

In [ ]:
prediction = predict(nbc, test, type= "prob")
head(prediction)